In [1]:
import datalabframework as dlf

In [2]:
dlf.version_info

(0, 5, 10)

In [3]:
engine = dlf.engines.get('spark')
spark = engine.context()

PYSPARK_SUBMIT_ARGS:  --packages mysql:mysql-connector-java:8.0.12 pyspark-shell


In [4]:
df = engine.read('.resources.actor')
df.printSchema()

{'column': 'last_update', 'end_date': '2006-02-16', 'policy': 'date', 'window': '1 days'}
start date 2006-02-15T00:00:00, end date 2006-02-16T00:00:00
filtered records 200
2018-10-19 07:17:45,662 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'actor', 'url': 'jdbc:mysql://mysql:3306/sakila'}
root
 |-- actor_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [5]:
df.show()

+--------+----------+---------+-------------------+
|actor_id|first_name|last_name|        last_update|
+--------+----------+---------+-------------------+
|     105|    SIDNEY|    CROWE|2006-02-15 04:34:33|
|     172|   GROUCHO| WILLIAMS|2006-02-15 04:34:33|
|      74|     MILLA|   KEITEL|2006-02-15 04:34:33|
|      48|   FRANCES|DAY-LEWIS|2006-02-15 04:34:33|
|      65|    ANGELA|   HUDSON|2006-02-15 04:34:33|
|      69|   KENNETH|  PALTROW|2006-02-15 04:34:33|
|      28|     WOODY|  HOFFMAN|2006-02-15 04:34:33|
|     104|  PENELOPE|   CRONYN|2006-02-15 04:34:33|
|     146|    ALBERT|JOHANSSON|2006-02-15 04:34:33|
|     198|      MARY|   KEITEL|2006-02-15 04:34:33|
|     182|    DEBBIE|   AKROYD|2006-02-15 04:34:33|
|      81|  SCARLETT|    DAMON|2006-02-15 04:34:33|
|      68|       RIP|  WINSLET|2006-02-15 04:34:33|
|      87|   SPENCER|     PECK|2006-02-15 04:34:33|
|      21|   KIRSTEN|  PALTROW|2006-02-15 04:34:33|
|     197|     REESE|     WEST|2006-02-15 04:34:33|
|     158|  

In [10]:
md = dlf.params.metadata()
for resource in md['resources']:
    ds = md['resources'][resource]
    if ds['provider']=='source':
        print(ds['path'])
        # read
        engine.read(resource).show(5)

actor
{'column': 'last_update', 'end_date': '2014-09-26', 'policy': 'date', 'window': '1 days'}
filter by date {'column': 'last_update', 'end_date': '2014-09-26', 'policy': 'date', 'window': '1 days'}
2014-09-25T00:00:00 2014-09-26T00:00:00
last_update < "2014-09-26T00:00:00" and last_update >= "2014-09-25T00:00:00" 
selected:  0
2018-10-19 05:05:34,195 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'actor', 'url': 'jdbc:mysql://mysql:3306/sakila'}
+--------+----------+---------+-----------+
|actor_id|first_name|last_name|last_update|
+--------+----------+---------+-----------+
+--------+----------+---------+-----------+

address
{'column': 'last_update', 'end_date': '2014-09-26', 'policy': 'date', 'window': '1 days'}
filter by date {'column': 'last_update', 'end_date': '2014-09-26', 'policy': 'date', 'window': '1 days'}
2014-09-25T00:00:00 2014-09-26T00:00:00
last_update < "2014-09-26T00:00:00" and 

In [ ]:
day = md = dlf.params.metadata()
for resource in md['resources']:
    ds = md['resources'][resource]
    if ds['provider']=='source':
        print(ds['path'])
        # read
        df_src = engine.read(resource)
        
        # define target path
        target_path = '{}.{}'.format(ds['provider'],ds['path'])
        print(target_path)
        
        # write
        engine.write(df_src, path=target_path, provider='target', mode='overwrite')

### Ingest from source to target 

In [1]:
import datalabframework as dlf

In [2]:
dlf.data.metadata('resources.city')
engine = dlf.engines.get('spark')
spark = engine.context()

PYSPARK_SUBMIT_ARGS:  --packages mysql:mysql-connector-java:8.0.12 pyspark-shell


In [3]:
# df =  engine.read('resources.actor')
# df.groupby('last_update').count().show()

In [9]:
engine.ingest(src_resource='resources.city', dest_path='new2.city', dest_provider='target')

{'column': 'last_update', 'end_date': '2006-02-16', 'policy': 'date', 'window': '1 days'}
start date 2006-02-15T00:00:00, end date 2006-02-16T00:00:00
filtered records 600
2018-10-19 08:04:48,585 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'city', 'url': 'jdbc:mysql://mysql:3306/sakila'}
None
2018-10-19 08:04:48,817 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'new2.city/schema', 'url': 'hdfs://hdfs-nn:8020/data/ingest/new2.city/schema'}
None
2018-10-19 08:04:49,146 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'new2.city/2018-10-19T080431', 'url': 'hdfs://hdfs-nn:8020/data/ingest/new2.city/2018-10-19T080431'}
{"fields":[{"metadata":{},"name":"city_id","nullable":true,"type":"integer"},{"metadata":{},"name":"city","nullab

In [16]:
engine.read(path='new2.city/schema', provider='target').show(truncate=False)

None
2018-10-19 08:05:48,712 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'new2.city/schema', 'url': 'hdfs://hdfs-nn:8020/data/ingest/new2.city/schema'}
+-----------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id               |date                      |schema                                                                                                                                                                                                                                                                                                         

In [17]:
df = engine.read(path='new2.city/2018-10-19T080533', provider='target')
df.count()

None
2018-10-19 08:06:01,330 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'new2.city/2018-10-19T080533', 'url': 'hdfs://hdfs-nn:8020/data/ingest/new2.city/2018-10-19T080533'}


1800

In [38]:
[x for x in df.columns if x != 'ingest_date']

['city_id', 'city', 'country_id', 'last_update', 'ohoh']

In [14]:
import pyspark.sql.functions as F
df = df.withColumn('ohoh', F.lit(42))
df = df.drop('ingest_date')

engine.write(df, path='test', provider='target', mode='overwrite')
engine.read(path='test', provider='target').show()

2018-10-19 08:05:20,230 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.write - {'format': 'parquet', 'service': 'hdfs', 'path': 'test', 'url': 'hdfs://hdfs-nn:8020/data/ingest/test'}
None
2018-10-19 08:05:20,456 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'test', 'url': 'hdfs://hdfs-nn:8020/data/ingest/test'}
+-------+-----------------+----------+-------------------+----+
|city_id|             city|country_id|        last_update|ohoh|
+-------+-----------------+----------+-------------------+----+
|    144|   Dhule (Dhulia)|        44|2006-02-15 04:45:25|  42|
|    442|      Saint-Denis|        79|2006-02-15 04:45:25|  42|
|    179|         Gatineau|        20|2006-02-15 04:45:25|  42|
|    560|            Vaduz|        55|2006-02-15 04:45:25|  42|
|    189|           Guaruj|        15|2006-02-15 04:45:25|  42|
|      2|             Abha|        82|2006-02-15 04:45

In [15]:
engine.ingest(src_path='test', src_provider='target', dest_path='new2.city', dest_provider='target')

None
2018-10-19 08:05:34,120 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'test', 'url': 'hdfs://hdfs-nn:8020/data/ingest/test'}
None
2018-10-19 08:05:34,348 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'new2.city/schema', 'url': 'hdfs://hdfs-nn:8020/data/ingest/new2.city/schema'}
None
2018-10-19 08:05:34,684 - INFO - 52e0f2d - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'new2.city/2018-10-19T080431', 'url': 'hdfs://hdfs-nn:8020/data/ingest/new2.city/2018-10-19T080431'}
{"fields":[{"metadata":{},"name":"city_id","nullable":true,"type":"integer"},{"metadata":{},"name":"city","nullable":true,"type":"string"},{"metadata":{},"name":"country_id","nullable":true,"type":"integer"},{"metadata":{},"name":"last_update","nullable":true,"type":"times

## pre checks

schema checks:

 - get table schema from source
 - compare with reference schema
 
value checks:
 - null
 - invalid 

## Ingest strategy

#### **what to read?**
 
FULL scan:  
tables are lost or never ingested before
 
  - cold start
  - disaster recovery from source
     
INCREMENTAL scan:  
We need a way to select/filter new data

  - time based (default, i.e. last_update column)
  - index based (if index is auto incrementing)
  - hash compare indexes (expensive)
  - full rescan and tag with ingest date
 
#### **where to write?**

  - define a naming convention for the target tables (default schema version: `latest`)  
    `<source>/<db-name>/<table-name>/<schema-version-date>`
    
#### **exceptions**
: what to do? (show error)

### HDFS

In [5]:
# read back from hdfs in parquet format
df_trg = engine.read('target', 'ingest')
df_trg.where(col(colname) => 'datetime')
df_trg.show()

hdfs://hdfs-nn:8020//data/ingest/extract/sakila.actor
+--------+----------+------------+-------------------+
|actor_id|first_name|   last_name|        last_update|
+--------+----------+------------+-------------------+
|       1|  PENELOPE|     GUINESS|2006-02-15 04:34:33|
|       2|      NICK|    WAHLBERG|2006-02-15 04:34:33|
|       3|        ED|       CHASE|2006-02-15 04:34:33|
|       4|  JENNIFER|       DAVIS|2006-02-15 04:34:33|
|       5|    JOHNNY|LOLLOBRIGIDA|2006-02-15 04:34:33|
|       6|     BETTE|   NICHOLSON|2006-02-15 04:34:33|
|       7|     GRACE|      MOSTEL|2006-02-15 04:34:33|
|       8|   MATTHEW|   JOHANSSON|2006-02-15 04:34:33|
|       9|       JOE|       SWANK|2006-02-15 04:34:33|
|      10| CHRISTIAN|       GABLE|2006-02-15 04:34:33|
|      11|      ZERO|        CAGE|2006-02-15 04:34:33|
|      12|      KARL|       BERRY|2006-02-15 04:34:33|
|      13|       UMA|        WOOD|2006-02-15 04:34:33|
|      14|    VIVIEN|      BERGEN|2006-02-15 04:34:33|
|      15| 

## post checks

In [6]:
assert(df_src.subtract(df_trg).count()==0)

In [ ]:
engine.read(path='abcd', provider='dsds')
write(df, path='abcd', provider='ingest')

engine.read('resource_alias')
engine.write(df, 'resource_alias')
